In [ ]:
import yfinance as yf
import pandas as pd
import warnings

# Silencia os alertas do yfinance
warnings.simplefilter(action='ignore', category=FutureWarning)

acoes = ['ITUB3.SA', 'CMIG4.SA', 'CSMG3.SA', 'BBSE3.SA', 'VIVT3.SA']

# Dicionário para guardar cada DataFrame separado
dados_acoes = {}

for acao in acoes:
    # Baixa os dados
    dados = yf.download(acao, start='2024-06-01', end='2024-06-28', progress=False)

    # Garante que só a coluna 'Close' seja usada
    df = dados[['Close']].copy()
    df.reset_index(inplace=True)
    df.rename(columns={'Close': 'Preço de Fechamento (R$)'}, inplace=True)
    df['Ação'] = acao.replace('.SA', '')

    # Salva no dicionário com nome da ação
    dados_acoes[acao.replace('.SA', '')] = df

# Exibe os 5 últimos de cada ação
for nome, df in dados_acoes.items():
    print(f"\n🔹 Últimos preços de {nome}:")
    print(df.tail())


## Todas as colunas da yFinance

In [ ]:
import yfinance as yf
import pandas as pd
import warnings

# Silencia os alertas do yfinance
warnings.simplefilter(action='ignore', category=FutureWarning)

# Lista de ações que você quer analisar
acoes = ['ITUB3.SA']

# Dicionário para guardar cada DataFrame separado (agora com todas as colunas)
dados_acoes = {}

print(df.tail())

AttributeError: 'dict' object has no attribute 'tail'

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional
import warnings

# Configurações iniciais
warnings.simplefilter(action='ignore', category=FutureWarning)

def get_current_month_dates() -> tuple[str, str]:
    """Retorna as datas de início e fim do mês atual.
    
    Returns:
        Tuple com as datas no formato 'YYYY-MM-DD'.
    """
    hoje = datetime.now()
    inicio_mes = f"{hoje.year}-{hoje.month:02d}-01"
    fim_hoje = hoje.strftime('%Y-%m-%d')
    return inicio_mes, fim_hoje

def get_last_year_dates() -> tuple[str, str]:
    """Retorna as datas de início e fim do mesmo mês no ano anterior.
    
    Returns:
        Tuple com as datas no formato 'YYYY-MM-DD'.
    """
    hoje = datetime.now()
    last_year = hoje.year - 1
    inicio_mes = f"{last_year}-{hoje.month:02d}-01"
    fim_mes = f"{last_year}-{hoje.month:02d}-28"
    return inicio_mes, fim_mes

def fetch_stock_data(ticker: str, start_date: str, end_date: str) -> Optional[pd.DataFrame]:
    """Obtém dados históricos de uma ação para o período especificado.
    
    Args:
        ticker: Código da ação (ex.: 'ITUB3.SA').
        start_date: Data inicial no formato 'YYYY-MM-DD'.
        end_date: Data final no formato 'YYYY-MM-DD'.
    
    Returns:
        DataFrame com os dados históricos ou None se houver erro.
    """
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if df.empty:
            print(f"⚠️ Sem dados históricos para {ticker}")
            return None
        return df
    except Exception as e:
        print(f"❌ Erro ao obter dados históricos para {ticker}: {e}")
        return None

def calculate_stock_metrics(ticker: str, nome: str, inicio_mes: str, fim_hoje: str, 
                          inicio_ano_passado: str, fim_ano_passado: str) -> Optional[Dict]:
    """Calcula métricas financeiras para uma ação.
    
    Args:
        ticker: Código da ação (ex.: 'ITUB3.SA').
        nome: Nome da ação sem o sufixo '.SA'.
        inicio_mes: Data de início do mês atual.
        fim_hoje: Data final (hoje).
        inicio_ano_passado: Data de início do mesmo mês no ano anterior.
        fim_ano_passado: Data de fim do mesmo mês no ano anterior.
    
    Returns:
        Dicionário com métricas financeiras ou None se houver erro.
    """
    try:
        # Obtém informações do ticker
        yf_ticker = yf.Ticker(ticker)
        info = yf_ticker.info

        # Dados do mês atual
        df = fetch_stock_data(ticker, inicio_mes, fim_hoje)
        if df is None or df.empty:
            print(f"⚠️ Dados históricos vazios ou inválidos para {nome}")
            return None

        df = df.reset_index()
        required_columns = ['Date', 'Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            print(f"⚠️ Colunas necessárias ausentes para {nome}: {df.columns}")
            return None

        # Seleciona a última linha válida
        dados_dia = df.iloc[-1]

        # Preço de 12 meses atrás
        historico_anterior = fetch_stock_data(ticker, inicio_ano_passado, fim_ano_passado)
        preco_ano_passado = None
        if historico_anterior is not None and not historico_anterior.empty:
            historico_anterior = historico_anterior.reset_index()
            if 'Close' in historico_anterior.columns:
                preco_ano_passado = pd.to_numeric(historico_anterior['Close'].iloc[-1], errors='coerce')

        # Cálculos
        preco_fechamento = pd.to_numeric(dados_dia['Close'], errors='coerce')
        volume = pd.to_numeric(dados_dia['Volume'], errors='coerce')
        variacao_12m = (
            round((preco_fechamento - preco_ano_passado) / preco_ano_passado * 100, 2)
            if preco_ano_passado is not None and pd.notnull(preco_fechamento) and preco_ano_passado != 0 else None
        )

        return {
            'ticker': nome,
            'data_cotacao': dados_dia['Date'],
            'preco_fechamento': round(float(preco_fechamento), 2) if pd.notnull(preco_fechamento) else None,
            'dividend_yield_12m': round(float(info.get('dividendYield', 0)) * 100, 2) if info.get('dividendYield') else None,
            'dividend_yield_5y': round(float(info.get('fiveYearAvgDividendYield', 0)) * 100, 2) if info.get('fiveYearAvgDividendYield') else None,
            'dividendos_12m': round(float(info.get('dividendRate', 0)), 2) if info.get('dividendRate') else None,
            'p_l': round(float(info.get('trailingPE', 0)), 2) if info.get('trailingPE') else None,
            'p_vp': round(float(info.get('priceToBook', 0)), 2) if info.get('priceToBook') else None,
            'roe': round(float(info.get('returnOnEquity', 0)) * 100, 2) if info.get('returnOnEquity') else None,
            'margem_liquida': round(float(info.get('netMargins', 0)) * 100, 2) if info.get('netMargins') else None,
            'divida_ebitda': round(float(info.get('debtToEquity', 0)), 2) if info.get('debtToEquity') else None,
            'variacao_12m': variacao_12m,
            'volume_negociado': int(volume) if pd.notnull(volume) else None
        }
    except Exception as e:
        print(f"❌ Erro ao calcular métricas para {nome}: {e}")
        return None

def main(acoes: List[str]) -> pd.DataFrame:
    """Função principal para processar dados financeiros de uma lista de ações.
    
    Args:
        acoes: Lista de tickers das ações.
    
    Returns:
        DataFrame com os dados consolidados.
    """
    dados_finais: List[Dict] = []
    inicio_mes, fim_hoje = get_current_month_dates()
    inicio_ano_passado, fim_ano_passado = get_last_year_dates()

    for acao in acoes:
        nome = acao.replace('.SA', '')
        metrics = calculate_stock_metrics(acao, nome, inicio_mes, fim_hoje, 
                                         inicio_ano_passado, fim_ano_passado)
        if metrics:
            dados_finais.append(metrics)

    # Converte para DataFrame
    df_final = pd.DataFrame(dados_finais)
    return df_final

if __name__ == "__main__":
    acoes = ['ITUB3.SA', 'CMIG4.SA', 'CSMG3.SA', 'BBSE3.SA', 'VIVT3.SA']
    df_final = main(acoes)

    # Exibe o resultado
    pd.set_option('display.max_columns', None)
    print("\n📊 Resultado consolidado:")
    print(df_final)

    # Salva em CSV
    if not df_final.empty:
        df_final.to_csv('dados_financeiros.csv', index=False, encoding='utf-8')
        print("\n✅ Dados salvos em 'dados_financeiros.csv'")
    else:
        print("\n⚠️ Nenhum dado válido para salvar.")

## Dividend yield

In [ ]:
import yfinance as yf
import pandas as pd
import warnings

# Silencia os alertas do yfinance
warnings.simplefilter(action='ignore', category=FutureWarning)

acao_ticker = 'CSMG3.SA'

# 1. Obter o objeto Ticker para a ação
acao = yf.Ticker(acao_ticker)

# 2. Obter informações gerais da ação (que pode incluir o Dividend Yield)
# Às vezes, o 'info' já traz o 'dividendYield' ou 'trailingAnnualDividendYield'
info_acao = acao.info
dividend_yield_yfinance_info = info_acao.get('dividendYield') # ou 'trailingAnnualDividendYield'
# O valor retornado é em decimal (ex: 0.05 para 5%)
if dividend_yield_yfinance_info is not None:
    print(f"🔹 Dividend Yield (do info) de {acao_ticker.replace('.SA', '')}: {dividend_yield_yfinance_info:.2%}")
else:
    print(f"🔹 Dividend Yield (do info) de {acao_ticker.replace('.SA', '')}: Não disponível diretamente.")

# 3. Obter o histórico de dividendos pagos
# Isso retornará um Series do pandas com as datas e os valores dos dividendos
dividendos = acao.dividends

print(f"\n🔹 Últimos dividendos de {acao_ticker.replace('.SA', '')}:")
print(dividendos.tail())

# 4. Obter o preço de fechamento atual ou um preço de referência
# Vamos usar o último preço de fechamento da sua base de dados
dados_historicos = yf.download(acao_ticker, start='2024-06-01', end='2024-06-28', progress=False)
ultimo_preco_fechamento = dados_historicos['Close'].iloc[-1]

print(f"\n🔹 Último preço de fechamento de {acao_ticker.replace('.SA', '')}: R${ultimo_preco_fechamento:.2f}")

# 5. Calcular o Dividend Yield TTM (Trailing Twelve Months - últimos 12 meses)
# Vamos somar os dividendos dos últimos 12 meses.
# Primeiro, filtre os dividendos para o período desejado (últimos 12 meses a partir da última data de dados)
data_fim_para_calculo = dados_historicos.index.max()
data_inicio_para_calculo = data_fim_para_calculo - pd.DateOffset(years=1)

dividendos_ultimos_12m = dividendos[(dividendos.index > data_inicio_para_calculo) &
                                     (dividendos.index <= data_fim_para_calculo)]

soma_dividendos_ultimos_12m = dividendos_ultimos_12m.sum()

if ultimo_preco_fechamento > 0:
    dividend_yield_calculado = (soma_dividendos_ultimos_12m / ultimo_preco_fechamento)
    print(f"\n🔹 Dividend Yield (calculado por você) de {acao_ticker.replace('.SA', '')}: {dividend_yield_calculado:.2%}")
else:
    print(f"\n🔹 Não foi possível calcular o Dividend Yield: Preço de fechamento inválido.")

### Biblioteca b3api

In [5]:
import yfinance as yf
import pandas as pd

# Lista de tickers da B3 (adicionar ".SA")
tickers = ["PETR4.SA", "ITUB4.SA", "VALE3.SA"]

# Dicionário para armazenar os dados
dados_acoes = {}

# Loop para buscar informações de cada ticker
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        dados_acoes[ticker] = {
            "Empresa": info.get("longName"),
            "Setor": info.get("sector"),
            "Preço Atual": info.get("currentPrice"),
            "P/L (TTM)": info.get("trailingPE"),
            "Dividend Yield (%)": info.get("dividendYield", 0) * 100, # Convertendo para percentual
            "Volume Médio": info.get("averageVolume"),
            "Beta": info.get("beta"),
            "Máxima 52 Semanas": info.get("fiftyTwoWeekHigh"),
            "Mínima 52 Semanas": info.get("fiftyTwoWeekLow")
        }
        print(f"Dados de {ticker} obtidos com sucesso.")
    except Exception as e:
        print(f"Não foi possível obter dados para {ticker}. Erro: {e}")

# Criar um DataFrame com os resultados
df_resultado = pd.DataFrame.from_dict(dados_acoes, orient="index")

# Exibir o DataFrame
print("\n--- Tabela Comparativa de Indicadores ---")
print(df_resultado.to_string()) # Usar to_string() para melhor visualização de todas as colunas

Dados de PETR4.SA obtidos com sucesso.
Dados de ITUB4.SA obtidos com sucesso.
Dados de VALE3.SA obtidos com sucesso.

--- Tabela Comparativa de Indicadores ---
                                       Empresa               Setor  Preço Atual  P/L (TTM)  Dividend Yield (%)  Volume Médio   Beta  Máxima 52 Semanas  Mínima 52 Semanas
PETR4.SA  Petróleo Brasileiro S.A. - Petrobras              Energy        31.21   8.504086              1637.0      46220003  0.318              40.76           28.86000
ITUB4.SA            Itaú Unibanco Holding S.A.  Financial Services        36.27   9.276215                58.0      23558370  0.314              38.62           27.10909
VALE3.SA                             Vale S.A.     Basic Materials        53.00   7.010582              1418.0      20961057  1.051              65.35           48.77000


In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Lista de tickers da B3 (adicionar ".SA")
# Você pode adicionar quantos tickers quiser aqui
tickers = ["PETR4.SA", "ITUB4.SA", "VALE3.SA", "WEGE3.SA", "MGLU3.SA"]

# Dicionário para armazenar os dados
dados_acoes = {}

print("Iniciando a coleta de dados...")

# Loop para buscar informações de cada ticker
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # --- Coletando os dados ---
        # Usamos .get(chave, "N/A") para evitar erros se uma chave não existir
        dados_acoes[ticker] = {
            "Empresa": info.get("longName", "N/A"),
            "Setor": info.get("sector", "N/A"),
            # NOVO: Data da cotação (convertendo o timestamp)
            "Data Cotação": datetime.fromtimestamp(info.get("regularMarketTime", 0)).strftime('%Y-%m-%d %H:%M:%S'),
            "Preço Atual": info.get("currentPrice", 0),
            # NOVO: Preço de Fechamento anterior
            "Preço Fechamento": info.get("previousClose", 0),
            # NOVO: Volume do dia
            "Volume Negociado": info.get("volume", 0),
            # --- Indicadores de Valuation ---
            "P/L": info.get("trailingPE", 0),
            # NOVO: Preço sobre Valor Patrimonial
            "P/VP": info.get("priceToBook", 0),
            # --- Indicadores de Dividendos ---
            "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
            # NOVO: Dividend Yield médio dos últimos 5 anos (%)
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", 0), # Já vem em formato percentual
            # NOVO: Dividendos por ação dos últimos 12 meses
            "Dividendos 12m (R$)": info.get("trailingAnnualDividendRate", 0),
            # --- Indicadores de Rentabilidade e Margem ---
            # NOVO: Retorno sobre o Patrimônio Líquido
            "ROE (%)": info.get("returnOnEquity", 0) * 100,
            # NOVO: Margem Líquida
            "Margem Líquida (%)": info.get("profitMargins", 0) * 100,
            # --- Indicadores de Endividamento e Performance ---
            # NOVO: Dívida Líquida / EBITDA (pode não estar disponível para todas)
            "Dívida/EBITDA": info.get("debtToEbitda", "N/A"),
            # NOVO: Variação da ação em 12 meses
            "Variação 12m (%)": info.get("52WeekChange", 0) * 100,
        }
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")

print("\nColeta de dados finalizada.")

# --- Organização e Exibição dos Dados ---
if dados_acoes:
    # Criar um DataFrame com os resultados
    df_resultado = pd.DataFrame.from_dict(dados_acoes, orient="index")

    # Formatando colunas numéricas para melhor visualização (2 casas decimais)
    colunas_para_formatar = [
        "Preço Atual", "Preço Fechamento", "P/L", "P/VP", "Dividend Yield (%)",
        "DY 5 Anos Média (%)", "Dividendos 12m (R$)", "ROE (%)",
        "Margem Líquida (%)", "Variação 12m (%)"
    ]
    for col in colunas_para_formatar:
        # Verifica se a coluna existe antes de tentar formatar
        if col in df_resultado.columns:
            df_resultado[col] = df_resultado[col].apply(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)


    # Exibir o DataFrame
    print("\n--- Tabela Comparativa de Indicadores Fundamentalistas ---")
    print(df_resultado.to_string())
else:
    print("Nenhum dado foi coletado.")

Iniciando a coleta de dados...
✅ Dados de PETR4.SA obtidos com sucesso.
✅ Dados de ITUB4.SA obtidos com sucesso.
✅ Dados de VALE3.SA obtidos com sucesso.
✅ Dados de WEGE3.SA obtidos com sucesso.
✅ Dados de MGLU3.SA obtidos com sucesso.

Coleta de dados finalizada.

--- Tabela Comparativa de Indicadores Fundamentalistas ---
                                       Empresa               Setor         Data Cotação Preço Atual Preço Fechamento  Volume Negociado    P/L  P/VP Dividend Yield (%) DY 5 Anos Média (%) Dividendos 12m (R$) ROE (%) Margem Líquida (%) Dívida/EBITDA Variação 12m (%)
PETR4.SA  Petróleo Brasileiro S.A. - Petrobras              Energy  2025-06-27 17:07:37       31.21            31.46          16394700   8.50  1.02            1637.00               20.27                5.60   12.02               9.70           N/A           -19.21
ITUB4.SA            Itaú Unibanco Holding S.A.  Financial Services  2025-06-27 17:07:44       36.27            36.31          13803900   9.28  1.

###

## Junção do csv para o codigo completo

In [11]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
# Nome do arquivo gerado pelo script anterior.
# Garanta que este arquivo esteja na mesma pasta que o script.
arquivo_input = r"E:\Github\finance-manager\datasets\scanner_simplificado_b3.xlsx"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_excel(arquivo_input)
    # Pega a lista de tickers da coluna 'Ticker' do seu arquivo.
    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ Sucesso. {len(tickers_para_analisar)} ativos serão analisados.")
except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_input}' não foi encontrado.")
    print("Por favor, execute o script anterior primeiro ou verifique o nome do arquivo.")
    # Encerra o script se o arquivo de entrada não existir.
    exit()
except Exception as e:
    print(f"❌ ERRO: Ocorreu um problema ao ler o arquivo: {e}")
    exit()


# --- PASSO 2: BUSCAR OS DADOS DETALHADOS PARA CADA TICKER DA LISTA ---
# Dicionário para armazenar os novos dados detalhados
dados_detalhados_acoes = {}

print("\nIniciando a coleta de dados detalhados...")

# Loop para buscar informações de cada ticker da sua lista
for ticker in tickers_para_analisar:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Combina as informações originais do seu arquivo com os novos dados
        # Primeiro, pegamos a linha original correspondente ao ticker
        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]

        dados_detalhados_acoes[ticker] = {
            # Colunas do seu arquivo original
            "Empresa": info_original['Empresa'],
            "Categoria": info_original['Categoria'],
            "País": info_original['País'],
            "Market Cap (R$)": info_original['Market Cap'],
            
            # Novas colunas detalhadas
            "Data Cotação": datetime.fromtimestamp(info.get("regularMarketTime", 0)).strftime('%Y-%m-%d %H:%M:%S'),
            "Preço Atual": info.get("currentPrice", 0),
            "Preço Fechamento": info.get("previousClose", 0),
            "Volume Negociado": info.get("volume", 0),
            "P/L": info.get("trailingPE", "N/A"),
            "P/VP": info.get("priceToBook", "N/A"),
            "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", "N/A"),
            "Dividendos 12m (R$)": info.get("trailingAnnualDividendRate", "N/A"),
            "ROE (%)": info.get("returnOnEquity", 0) * 100,
            "Margem Líquida (%)": info.get("profitMargins", 0) * 100,
            "Dívida/EBITDA": info.get("debtToEbitda", "N/A"),
            "Variação 12m (%)": info.get("52WeekChange", 0) * 100,
        }
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")

print("\nColeta de dados finalizada.")

# --- PASSO 3: ORGANIZAR E SALVAR O NOVO RELATÓRIO DETALHADO ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")

    # Formatando colunas numéricas para melhor visualização (2 casas decimais)
    colunas_para_formatar = [
        "Preço Atual", "Preço Fechamento", "P/L", "P/VP", "Dividend Yield (%)",
        "DY 5 Anos Média (%)", "Dividendos 12m (R$)", "ROE (%)",
        "Margem Líquida (%)", "Variação 12m (%)"
    ]
    for col in colunas_para_formatar:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
            )

    # Exibir o DataFrame
    print("\n--- Tabela Comparativa de Indicadores Fundamentalistas ---")
    print(df_resultado_detalhado.to_string())

    # Salvar o novo relatório detalhado em um NOVO arquivo Excel
    try:
        df_resultado_detalhado.to_excel("relatorio_detalhado_b3.xlsx")
        print("\n✅ Relatório detalhado salvo com sucesso no arquivo 'relatorio_detalhado_b3.xlsx'")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o novo arquivo Excel: {e}")
else:
    print("\nNenhum dado foi coletado.")

Lendo a lista de ativos do arquivo 'E:\Github\finance-manager\datasets\scanner_simplificado_b3.xlsx'...
✅ Sucesso. 69 ativos serão analisados.

Iniciando a coleta de dados detalhados...
✅ Dados de ITUB4.SA obtidos com sucesso.
✅ Dados de ITUB3.SA obtidos com sucesso.
✅ Dados de ROXO34.SA obtidos com sucesso.
✅ Dados de BBDC4.SA obtidos com sucesso.
✅ Dados de BBDC3.SA obtidos com sucesso.
✅ Dados de SANB11.SA obtidos com sucesso.
✅ Dados de BBAS3.SA obtidos com sucesso.
✅ Dados de SANB4.SA obtidos com sucesso.
✅ Dados de SANB3.SA obtidos com sucesso.
✅ Dados de INBR32.SA obtidos com sucesso.
✅ Dados de BPAN4.SA obtidos com sucesso.
✅ Dados de BNBR3.SA obtidos com sucesso.
✅ Dados de ABCB4.SA obtidos com sucesso.
✅ Dados de BRSR5.SA obtidos com sucesso.
✅ Dados de BRSR3.SA obtidos com sucesso.
✅ Dados de BRSR6.SA obtidos com sucesso.
✅ Dados de BSLI4.SA obtidos com sucesso.
✅ Dados de BSLI3.SA obtidos com sucesso.
✅ Dados de BAZA3.SA obtidos com sucesso.
✅ Dados de BMEB3.SA obtidos com 